In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from collections import Counter, defaultdict
import random

class IALabo:
    """Système d'analyse intelligent pour jeux de hasard"""
    
    def __init__(self, fichier_donnees):
        """
        Initialise IA Labo
        :param fichier_donnees: Chemin vers le fichier historique
        """
        # Définir les attributs d'abord
        self.numero_max = 49  # Pour Loto français
        self.historique_scores = defaultdict(list)
        self.performance_reseaux = {}
        
        # Puis charger les données
        self.df = self.charger_donnees(fichier_donnees)
        
    def charger_donnees(self, fichier):
        """Charge et prépare les données historiques"""
        try:
            print(f"📖 Chargement du fichier: {fichier}")
            
            # Essayer différentes méthodes de chargement
            try:
                df = pd.read_csv(fichier, sep=';')
            except:
                df = pd.read_csv(fichier, sep=',')
            
            print(f"Colonnes détectées: {df.columns.tolist()}")
            print(f"Aperçu des données:\n{df.head()}")
            
            # Nettoyer les noms de colonnes
            df.columns = [col.strip().lower() for col in df.columns]
            
            # Identifier automatiquement les colonnes de numéros
            numeros_columns = []
            
            # Chercher des colonnes nommées num0, num1, etc.
            for i in range(5):
                col_name = f'num{i}'
                if col_name in df.columns:
                    numeros_columns.append(col_name)
            
            print(f"Colonnes de numéros identifiées: {numeros_columns}")
            
            # Si on n'a pas trouvé toutes les colonnes
            if len(numeros_columns) < 5:
                print("⚠️  Moins de 5 colonnes de numéros trouvées")
                # Prendre les premières colonnes numériques
                numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
                # Exclure la colonne date si elle est numérique
                if 'date' in numeric_cols:
                    numeric_cols.remove('date')
                numeros_columns = numeric_cols[:5]
            
            print(f"Colonnes finales utilisées: {numeros_columns}")
            
            # Renommer les colonnes pour uniformité
            for i, col in enumerate(numeros_columns[:5]):
                if f'num{i}' not in df.columns:
                    df[f'num{i}'] = pd.to_numeric(df[col], errors='coerce')
            
            # Garder seulement les 5 colonnes de numéros standardisées
            numeros_columns_standard = [f'num{i}' for i in range(5)]
            df = df[numeros_columns_standard].copy()
            
            # Supprimer les lignes avec valeurs manquantes
            df = df.dropna()
            
            # Convertir en entiers
            for col in numeros_columns_standard:
                df[col] = df[col].astype(int)
            
            # Vérifier que les numéros sont dans la bonne plage
            for col in numeros_columns_standard:
                df = df[(df[col] >= 1) & (df[col] <= self.numero_max)]
            
            # Trier les numéros dans l'ordre croissant
            df[numeros_columns_standard] = np.sort(df[numeros_columns_standard].values, axis=1)
            
            print(f"✅ {len(df)} tirages valides chargés")
            print(f"Plage des numéros: {df.min().min()} à {df.max().max()}")
            
            return df
            
        except Exception as e:
            print(f"❌ Erreur lors du chargement: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    # ==================== MODULES D'ANALYSE SIMPLIFIÉS ====================
    
    class AnalyseFrequentielle:
        """Module 1: Analyse fréquentielle"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des fréquences"""
            scores = np.zeros(49)
            
            # Compter les apparitions
            compteur = Counter()
            for tirage in self.historique:
                compteur.update(tirage)
            
            total_tirages = len(self.historique)
            if total_tirages > 0:
                for num, count in compteur.items():
                    if 1 <= num <= 49:
                        scores[num-1] = count / total_tirages
            
            return scores
    
    class AnalyseRetards:
        """Module 2: Analyse des retards"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des numéros en retard"""
            scores = np.zeros(49)
            
            # Calculer le délai depuis la dernière apparition
            derniere_apparition = {i: -1000 for i in range(1, 50)}  # Initialiser avec une valeur basse
            
            for idx, tirage in enumerate(self.historique):
                for num in tirage:
                    derniere_apparition[num] = idx
            
            max_retard = max(idx for idx in derniere_apparition.values())
            
            for num, dernier_idx in derniere_apparition.items():
                retard = len(self.historique) - dernier_idx
                scores[num-1] = retard / max_retard if max_retard > 0 else 0
            
            return scores
    
    class AnalysePairsImpairs:
        """Module 3: Analyse pairs/impairs"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse de la répartition pairs/impairs"""
            scores = np.zeros(49)
            
            if not self.historique:
                return scores
            
            # Analyser la tendance récente
            derniers_tirages = self.historique[-20:] if len(self.historique) > 20 else self.historique
            
            # Compter les pairs et impairs
            total_numeros = sum(len(t) for t in derniers_tirages)
            pairs_recent = sum(1 for tirage in derniers_tirages for num in tirage if num % 2 == 0)
            ratio_pairs = pairs_recent / total_numeros if total_numeros > 0 else 0.5
            
            # Attribuer des scores selon le ratio
            for i in range(1, 50):
                if i % 2 == 0:  # Pair
                    scores[i-1] = ratio_pairs
                else:  # Impair
                    scores[i-1] = 1 - ratio_pairs
            
            return scores
    
    class AnalyseSommes:
        """Module 4: Analyse des sommes"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des sommes moyennes"""
            scores = np.zeros(49)
            
            if not self.historique:
                return scores
            
            # Calculer la somme moyenne des derniers tirages
            sommes = [sum(t) for t in self.historique[-30:]]
            if not sommes:
                return scores
                
            somme_moyenne = np.mean(sommes)
            somme_std = np.std(sommes) if len(sommes) > 1 else 30
            
            # Score basé sur la contribution à une somme proche de la moyenne
            contribution_ideale = somme_moyenne / 5
            
            for i in range(1, 50):
                distance = abs(i - contribution_ideale)
                scores[i-1] = np.exp(-(distance**2) / (2 * (somme_std/5)**2))
            
            return scores
    
    class AnalyseClusters:
        """Module 5: Analyse des clusters"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des clusters de numéros"""
            scores = np.zeros(49)
            
            if len(self.historique) < 10:
                return scores
            
            # Analyser les paires fréquentes
            paires = Counter()
            for tirage in self.historique:
                for i in range(len(tirage)):
                    for j in range(i+1, len(tirage)):
                        paire = tuple(sorted([tirage[i], tirage[j]]))
                        paires[paire] += 1
            
            if paires:
                max_freq = max(paires.values()) if paires.values() else 1
                for (num1, num2), freq in paires.items():
                    score = freq / max_freq
                    scores[num1-1] += score * 0.1
                    scores[num2-1] += score * 0.1
            
            return scores
    
    class AnalyseDizaines:
        """Module 6: Analyse par dizaines"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse de la répartition par dizaines"""
            scores = np.zeros(49)
            
            if not self.historique:
                return scores
            
            # Analyser la fréquence par dizaine
            compteur_dizaines = {d: 0 for d in range(5)}  # 0:1-9, 1:10-19, etc.
            total_numeros = 0
            
            derniers_tirages = self.historique[-50:] if len(self.historique) > 50 else self.historique
            for tirage in derniers_tirages:
                for num in tirage:
                    dizaine = (num - 1) // 10
                    if dizaine < 5:  # S'assurer que c'est valide
                        compteur_dizaines[dizaine] += 1
                        total_numeros += 1
            
            if total_numeros > 0:
                for num in range(1, 50):
                    dizaine = (num - 1) // 10
                    if dizaine < 5:
                        freq_dizaine = compteur_dizaines[dizaine] / total_numeros
                        scores[num-1] = 1 - freq_dizaine  # Favorise les dizaines sous-représentées
            
            return scores
    
    # ==================== MOTEUR D'AGRÉGATION ====================
    
    def analyser_avec_tous_modules(self):
        """Exécute tous les modules d'analyse"""
        if self.df is None or len(self.df) < 10:
            print("❌ Pas assez de données historiques")
            return None
        
        # Préparer les données historiques
        numeros_columns = [f'num{i}' for i in range(5)]
        historique_tirages = self.df[numeros_columns].values.tolist()
        
        print(f"📊 Analyse de {len(historique_tirages)} tirages historiques...")
        
        # Initialiser les modules
        modules = {
            'frequentiel': self.AnalyseFrequentielle(historique_tirages),
            'retards': self.AnalyseRetards(historique_tirages),
            'pairs_impairs': self.AnalysePairsImpairs(historique_tirages),
            'sommes': self.AnalyseSommes(historique_tirages),
            'clusters': self.AnalyseClusters(historique_tirages),
            'dizaines': self.AnalyseDizaines(historique_tirages)
        }
        
        # Exécuter toutes les analyses
        resultats = {}
        for nom, module in modules.items():
            scores = module.analyser()
            resultats[nom] = scores
            print(f"   ✅ Module {nom}")
        
        return resultats
    
    def bagging(self, resultats_modules):
        """Méthode Bagging: moyenne simple des scores"""
        if not resultats_modules:
            return None
        
        # Moyenne des scores de tous les modules
        scores_combined = np.zeros(self.numero_max)
        
        for module_name, scores in resultats_modules.items():
            scores_combined += scores
        
        scores_combined /= len(resultats_modules)
        
        # Normaliser entre 0 et 1
        if scores_combined.max() > 0:
            scores_combined = scores_combined / scores_combined.max()
        
        # Classement des numéros
        classement = np.argsort(scores_combined)[::-1] + 1
        
        return {
            'scores': scores_combined,
            'classement': classement[:20],  # Top 20
            'methode': 'bagging'
        }
    
    def boosting(self, resultats_modules):
        """Méthode Boosting: pondération intelligente"""
        if not resultats_modules:
            return None
        
        # Pondérations selon le type d'analyse
        poids = {
            'frequentiel': 1.0,    # Fréquences historiques
            'retards': 1.2,        # Retards (important)
            'pairs_impairs': 0.8,  # Équilibre pairs/impairs
            'sommes': 1.1,         # Sommes moyennes
            'clusters': 0.9,       # Paires fréquentes
            'dizaines': 1.0        # Répartition
        }
        
        # Application des poids
        scores_combined = np.zeros(self.numero_max)
        poids_total = sum(poids.values())
        
        for module_name, scores in resultats_modules.items():
            poids_module = poids.get(module_name, 1.0)
            scores_combined += scores * poids_module
        
        scores_combined /= poids_total
        
        # Normaliser
        if scores_combined.max() > 0:
            scores_combined = scores_combined / scores_combined.max()
        
        # Classement
        classement = np.argsort(scores_combined)[::-1] + 1
        
        return {
            'scores': scores_combined,
            'classement': classement[:20],
            'methode': 'boosting',
            'poids': poids
        }
    
    def generer_recommandations(self, methode='bagging'):
        """Génère des recommandations finales"""
        print(f"\n{'='*60}")
        print(f"🧠 IA LABO - Génération de recommandations")
        print(f"{'='*60}")
        
        # 1. Analyser avec tous les modules
        resultats_modules = self.analyser_avec_tous_modules()
        if resultats_modules is None:
            return
        
        # 2. Appliquer la méthode d'agrégation
        if methode == 'bagging':
            resultat = self.bagging(resultats_modules)
        elif methode == 'boosting':
            resultat = self.boosting(resultats_modules)
        else:
            resultat = self.bagging(resultats_modules)
        
        if resultat is None:
            return
        
        # 3. Afficher les résultats
        self.afficher_resultats(resultat, resultats_modules)
        
        # 4. Générer des combinaisons suggérées
        if len(resultat['classement']) >= 12:
            combinaisons = self.generer_combinaisons(resultat['classement'][:12])
            self.afficher_combinaisons(combinaisons)
        
        return resultat
    
    def afficher_resultats(self, resultat_aggrege, resultats_modules):
        """Affiche les résultats d'analyse"""
        print(f"\n📊 RÉSULTATS - Méthode: {resultat_aggrege['methode'].upper()}")
        
        if 'poids' in resultat_aggrege:
            print("\n⚖️  Pondérations des modules:")
            for module, poids in sorted(resultat_aggrege['poids'].items(), key=lambda x: x[1], reverse=True):
                print(f"   {module:15} → {poids:.3f}")
        
        print(f"\n🏆 TOP 15 NUMÉROS RECOMMANDÉS:")
        print("   Rang | Numéro | Score  | Détails")
        print("   " + "-" * 40)
        
        for i, num in enumerate(resultat_aggrege['classement'][:15], 1):
            score = resultat_aggrege['scores'][num-1]
            
            # Caractéristiques du numéro
            est_pair = "Pair" if num % 2 == 0 else "Impair"
            dizaine = (num - 1) // 10
            details = f"{est_pair}, Dizaine {dizaine*10+1}-{dizaine*10+10}"
            
            print(f"   {i:4} | {num:6} | {score:.4f} | {details}")
    
    def generer_combinaisons(self, numeros_recommandes, n_combinaisons=5):
        """Génère des combinaisons équilibrées"""
        combinaisons = []
        
        for _ in range(n_combinaisons):
            # Essayer plusieurs fois pour avoir une combinaison valide
            for attempt in range(100):
                # Sélection aléatoire
                combinaison = random.sample(numeros_recommandes.tolist(), 5)
                combinaison.sort()
                
                # Vérifier les critères
                if self.combinaison_valide(combinaison):
                    combinaisons.append(combinaison)
                    break
        
        return combinaisons[:n_combinaisons]
    
    def combinaison_valide(self, combinaison):
        """Vérifie si une combinaison respecte certains critères"""
        # 1. Au moins 1 pair et 1 impair
        pairs = sum(1 for n in combinaison if n % 2 == 0)
        if pairs == 0 or pairs == 5:
            return False
        
        # 2. Au moins 2 dizaines différentes
        dizaines = set((n-1)//10 for n in combinaison)
        if len(dizaines) < 2:
            return False
        
        # 3. Somme dans une plage raisonnable (100-200 pour 5/49)
        somme = sum(combinaison)
        if somme < 100 or somme > 200:
            return False
        
        return True
    
    def afficher_combinaisons(self, combinaisons):
        """Affiche les combinaisons suggérées"""
        print(f"\n🎯 {len(combinaisons)} COMBINAISONS SUGGÉRÉES:")
        print("   (Critères: équilibre pairs/impairs, diversité dizaines, somme 100-200)")
        
        for i, comb in enumerate(combinaisons, 1):
            somme = sum(comb)
            pairs = sum(1 for n in comb if n % 2 == 0)
            dizaines = len(set((n-1)//10 for n in comb))
            
            print(f"\n   Option {i}: {comb}")
            print(f"      • Somme: {somme} (moyenne: {somme/5:.1f})")
            print(f"      • Pairs/Impairs: {pairs}/{5-pairs}")
            print(f"      • Dizaines: {dizaines}/5")
            print(f"      • Écart: {max(comb)-min(comb)}")

# ==================== UTILISATION SIMPLIFIÉE ====================

if __name__ == "__main__":
    print("""
    ██╗ █████╗      ██╗      █████╗ ██████╗  ██████╗ 
    ██║██╔══██╗     ██║     ██╔══██╗██╔══██╗██╔═══██╗
    ██║███████║     ██║     ███████║██████╔╝██║   ██║
    ██║██╔══██║██   ██║     ██╔══██║██╔══██╗██║   ██║
    ██║██║  ██║╚█████╔╝     ██║  ██║██████╔╝╚██████╔╝
    ╚═╝╚═╝  ╚═╝ ╚════╝      ╚═╝  ╚═╝╚═════╝  ╚═════╝ 
    
    Système d'analyse intelligent pour jeux de hasard
    Version 1.0 - Recherche appliquée
    """)
    
    try:
        # Initialiser IA Labo
        print("\n📊 INITIALISATION DU SYSTÈME...")
        ia_labo = IALabo('lotodata1.csv')
        
        if ia_labo.df is not None and len(ia_labo.df) >= 10:
            # Générer des recommandations avec Bagging
            print("\n" + "="*60)
            print("1. ANALYSE AVEC BAGGING (MOYENNE SIMPLE)")
            print("="*60)
            resultats_bagging = ia_labo.generer_recommandations(methode='bagging')
            
            print("\n" + "="*60)
            print("2. ANALYSE AVEC BOOSTING (PONDÉRATION INTELLIGENTE)")
            print("="*60)
            resultats_boosting = ia_labo.generer_recommandations(methode='boosting')
            
            print("\n" + "="*60)
            print("📋 NOTE D'UTILISATION:")
            print("""
            IA Labo est un outil de recherche qui :
            • Combine 6 méthodes d'analyse différentes
            • Utilise l'agrégation pour réduire les biais
            • Propose des combinaisons statistiquement diversifiées
            • NE GARANTIT PAS de gains
            
            L'approche vise à créer des jeux plus intéressants
            statistiquement, pas à prédire l'avenir.
            """)
        else:
            print(f"\n❌ Impossible de continuer: {len(ia_labo.df) if ia_labo.df is not None else 0} tirages valides seulement")
            print("   Le système nécessite au moins 10 tirages historiques")
            
    except Exception as e:
        print(f"\n❌ ERREUR CRITIQUE: {e}")
        import traceback
        traceback.print_exc()


    ██╗ █████╗      ██╗      █████╗ ██████╗  ██████╗ 
    ██║██╔══██╗     ██║     ██╔══██╗██╔══██╗██╔═══██╗
    ██║███████║     ██║     ███████║██████╔╝██║   ██║
    ██║██╔══██║██   ██║     ██╔══██║██╔══██╗██║   ██║
    ██║██║  ██║╚█████╔╝     ██║  ██║██████╔╝╚██████╔╝
    ╚═╝╚═╝  ╚═╝ ╚════╝      ╚═╝  ╚═╝╚═════╝  ╚═════╝ 
    
    Système d'analyse intelligent pour jeux de hasard
    Version 1.0 - Recherche appliquée
    

📊 INITIALISATION DU SYSTÈME...
📖 Chargement du fichier: lotodata1.csv
Colonnes détectées: ['date', 'num0', 'num1', 'num2', 'num3', 'num4']
Aperçu des données:
         date  num0  num1  num2  num3  num4
0  24/09/2025     3    32    21    23    31
1  22/09/2025    10    41     5    29    47
2  20/09/2025    36    18    10    39    47
3  17/09/2025     2    25    47    32    37
4  15/09/2025    30     2    10    32    17
Colonnes de numéros identifiées: ['num0', 'num1', 'num2', 'num3', 'num4']
Colonnes finales utilisées: ['num0', 'num1', 'num2', 'num3', 'num4']
✅ 922 